# Session 17: Combining DataFrames and Data Quality

In the real world, data rarely comes in a single, clean file. You'll often need to combine data from multiple sources and clean up messy data. This session covers essential techniques for data integration and quality.

## Learning Objectives

By the end of this session, you will be able to:
- Combine DataFrames vertically with `concat`
- Join DataFrames horizontally with `merge`
- Understand different types of joins
- Convert data types with `astype`
- Detect and handle missing values
- Identify and handle outliers

In [ ]:
import pandas as pd
import numpy as np

## 1. Concatenation with concat()

Use `concat()` to stack DataFrames vertically (add more rows) or horizontally (add more columns).

### Vertical Concatenation (Stacking Rows)

In [ ]:
# Imagine we have sales data from different months
january_sales = pd.DataFrame({
    'product': ['Laptop', 'Phone', 'Tablet'],
    'units': [50, 120, 80],
    'revenue': [50000, 72000, 32000]
})

february_sales = pd.DataFrame({
    'product': ['Laptop', 'Phone', 'Tablet'],
    'units': [45, 150, 90],
    'revenue': [45000, 90000, 36000]
})

print("January Sales:")
display(january_sales)

print("\nFebruary Sales:")
display(february_sales)

In [ ]:
# Concatenate vertically (stack rows)
combined = pd.concat([january_sales, february_sales])
combined

In [ ]:
# Notice the index is duplicated! Reset it:
combined = pd.concat([january_sales, february_sales], ignore_index=True)
combined

In [ ]:
# Add keys to identify source
combined_keyed = pd.concat(
    [january_sales, february_sales], 
    keys=['January', 'February']
)
combined_keyed

In [ ]:
# Access data by key
combined_keyed.loc['January']

### Handling Mismatched Columns

In [ ]:
# What if DataFrames have different columns?
df1 = pd.DataFrame({
    'A': [1, 2],
    'B': [3, 4]
})

df2 = pd.DataFrame({
    'B': [5, 6],
    'C': [7, 8]
})

# By default, concat creates NaN for missing columns
combined = pd.concat([df1, df2], ignore_index=True)
combined

In [ ]:
# Use join='inner' to keep only common columns
combined_inner = pd.concat([df1, df2], ignore_index=True, join='inner')
combined_inner

### Horizontal Concatenation (Adding Columns)

In [ ]:
# Concatenate horizontally with axis=1
products = pd.DataFrame({
    'product_id': [1, 2, 3],
    'name': ['Laptop', 'Phone', 'Tablet']
})

prices = pd.DataFrame({
    'price': [1000, 600, 400],
    'cost': [700, 350, 250]
})

combined = pd.concat([products, prices], axis=1)
combined

## 2. Merging DataFrames

Use `merge()` to join DataFrames based on common columns (like SQL JOINs).

### Basic Merge

In [ ]:
# Products table
products = pd.DataFrame({
    'product_id': [1, 2, 3, 4],
    'name': ['Laptop', 'Phone', 'Tablet', 'Watch'],
    'category': ['Computers', 'Mobile', 'Mobile', 'Wearable']
})

# Orders table
orders = pd.DataFrame({
    'order_id': [101, 102, 103, 104, 105],
    'product_id': [1, 2, 1, 3, 2],
    'quantity': [1, 2, 1, 3, 1],
    'customer': ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve']
})

print("Products:")
display(products)

print("\nOrders:")
display(orders)

In [ ]:
# Merge on common column (product_id)
merged = pd.merge(orders, products, on='product_id')
merged

In [ ]:
# When column names differ, use left_on and right_on
products_renamed = products.rename(columns={'product_id': 'id'})

merged = pd.merge(
    orders, 
    products_renamed, 
    left_on='product_id', 
    right_on='id'
)
merged

## 3. Types of Joins

The `how` parameter controls how rows are matched:

- `'inner'` (default): Only matching rows from both DataFrames
- `'left'`: All rows from left DataFrame + matching from right
- `'right'`: All rows from right DataFrame + matching from left
- `'outer'`: All rows from both DataFrames

In [ ]:
# Create DataFrames with some non-matching values
employees = pd.DataFrame({
    'emp_id': [1, 2, 3, 4, 5],
    'name': ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve'],
    'dept_id': [10, 20, 10, 30, None]  # Eve has no department
})

departments = pd.DataFrame({
    'dept_id': [10, 20, 40],  # No department 30, has unused 40
    'dept_name': ['Engineering', 'Marketing', 'Finance']
})

print("Employees:")
display(employees)

print("\nDepartments:")
display(departments)

In [ ]:
# INNER JOIN: Only rows with matching dept_id in BOTH tables
inner = pd.merge(employees, departments, on='dept_id', how='inner')
print("INNER JOIN:")
inner

In [ ]:
# LEFT JOIN: All employees, with department info where available
left = pd.merge(employees, departments, on='dept_id', how='left')
print("LEFT JOIN:")
left

In [ ]:
# RIGHT JOIN: All departments, with employee info where available
right = pd.merge(employees, departments, on='dept_id', how='right')
print("RIGHT JOIN:")
right

In [ ]:
# OUTER JOIN: All rows from both tables
outer = pd.merge(employees, departments, on='dept_id', how='outer')
print("OUTER JOIN:")
outer

### Visual Summary of Joins

```
INNER: Only matching rows     LEFT: All from left + matches from right
+---+---+                      +---+---+
| A | B |                      | A | B |
+---+---+                      +---+---+
     ^^^                       ^^^ ^^^

RIGHT: All from right + matches from left    OUTER: All from both
+---+---+                                    +---+---+
| A | B |                                    | A | B |
+---+---+                                    +---+---+
    ^^^ ^^^                                  ^^^ ^^^ ^^^
```

### Merging on Multiple Columns

In [ ]:
# Sometimes you need to match on multiple columns
sales_q1 = pd.DataFrame({
    'year': [2023, 2023, 2024, 2024],
    'quarter': ['Q1', 'Q1', 'Q1', 'Q1'],
    'product': ['A', 'B', 'A', 'B'],
    'sales': [100, 200, 150, 250]
})

targets = pd.DataFrame({
    'year': [2023, 2023, 2024, 2024],
    'quarter': ['Q1', 'Q1', 'Q1', 'Q1'],
    'product': ['A', 'B', 'A', 'B'],
    'target': [120, 180, 140, 220]
})

# Merge on multiple columns
result = pd.merge(sales_q1, targets, on=['year', 'quarter', 'product'])
result['achievement'] = (result['sales'] / result['target'] * 100).round(1)
result

## 4. Data Types and astype()

Correct data types are essential for analysis. Use `astype()` to convert between types.

In [ ]:
# Create a DataFrame with mixed/incorrect types
data = pd.DataFrame({
    'id': ['1', '2', '3', '4'],  # Should be int
    'price': ['10.99', '20.50', '15.75', '8.25'],  # Should be float
    'quantity': [1.0, 2.0, 3.0, 1.0],  # Could be int
    'active': ['True', 'False', 'True', 'True']  # Should be bool
})

print("Original types:")
print(data.dtypes)
data

In [ ]:
# Convert individual columns
data['id'] = data['id'].astype(int)
data['price'] = data['price'].astype(float)
data['quantity'] = data['quantity'].astype(int)

print("After conversion:")
print(data.dtypes)

In [ ]:
# Boolean conversion (strings 'True'/'False' don't convert directly)
data['active'] = data['active'].map({'True': True, 'False': False})
print(data.dtypes)
data

In [ ]:
# Convert multiple columns at once
df = pd.DataFrame({
    'a': ['1', '2', '3'],
    'b': ['4', '5', '6'],
    'c': ['7.1', '8.2', '9.3']
})

df = df.astype({'a': int, 'b': int, 'c': float})
df.dtypes

In [ ]:
# Category type (memory efficient for repeated strings)
orders = pd.DataFrame({
    'order_id': range(1, 1001),
    'status': np.random.choice(['Pending', 'Shipped', 'Delivered'], 1000)
})

print(f"Memory before: {orders['status'].memory_usage(deep=True):,} bytes")

orders['status'] = orders['status'].astype('category')

print(f"Memory after: {orders['status'].memory_usage(deep=True):,} bytes")

## 5. Missing Values

Missing data is represented as `NaN` (Not a Number) in Pandas. Handling missing values correctly is crucial.

### Detecting Missing Values

In [ ]:
# Create a DataFrame with missing values
survey = pd.DataFrame({
    'name': ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve'],
    'age': [25, None, 35, 28, None],
    'salary': [50000, 60000, None, 55000, 48000],
    'department': ['IT', 'HR', 'IT', None, 'Finance']
})

survey

In [ ]:
# Check for missing values
print("Is null (True = missing):")
survey.isnull()

In [ ]:
# Count missing values per column
print("Missing values per column:")
survey.isnull().sum()

In [ ]:
# Percentage of missing values
print("Percentage missing:")
(survey.isnull().sum() / len(survey) * 100).round(1)

In [ ]:
# Total missing values in the DataFrame
print(f"Total missing: {survey.isnull().sum().sum()}")

In [ ]:
# Check for non-null values with notnull()
print("Rows where age is NOT null:")
survey[survey['age'].notnull()]

### Removing Missing Values

In [ ]:
# Drop rows with ANY missing value
clean = survey.dropna()
print("After dropping rows with any missing value:")
clean

In [ ]:
# Drop rows only if a specific column has missing values
clean_age = survey.dropna(subset=['age'])
print("After dropping rows where 'age' is missing:")
clean_age

In [ ]:
# Drop rows only if ALL values are missing
# (First, let's add a row that's all NaN)
survey_with_empty = pd.concat([survey, pd.DataFrame([[None]*4], columns=survey.columns)], ignore_index=True)
print("With empty row:")
display(survey_with_empty)

clean = survey_with_empty.dropna(how='all')
print("\nAfter dropping all-NaN rows:")
clean

In [ ]:
# Drop columns with missing values
clean_cols = survey.dropna(axis=1)
print("After dropping columns with any missing value:")
clean_cols

### Filling Missing Values

In [ ]:
# Fill with a constant value
filled = survey.fillna('Unknown')
print("After filling with 'Unknown':")
filled

In [ ]:
# Fill different columns with different values
filled = survey.fillna({
    'age': survey['age'].median(),
    'salary': survey['salary'].mean(),
    'department': 'Unknown'
})
print("After filling with appropriate values:")
filled

In [ ]:
# Forward fill (use previous row's value)
time_series = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=7),
    'value': [100, None, None, 110, None, 115, 120]
})

print("Original:")
display(time_series)

time_series['value_ffill'] = time_series['value'].ffill()
print("\nAfter forward fill:")
time_series

In [ ]:
# Backward fill (use next row's value)
time_series['value_bfill'] = time_series['value'].bfill()
print("After backward fill:")
time_series

## 6. Detecting and Handling Outliers

Outliers are extreme values that can skew your analysis. Let's learn how to detect and handle them.

In [ ]:
# Create a dataset with outliers
np.random.seed(42)
salaries = pd.DataFrame({
    'employee_id': range(1, 101),
    'salary': np.concatenate([
        np.random.normal(50000, 10000, 95),  # Normal salaries
        [200000, 5000, 180000, 3000, 250000]  # Outliers
    ])
})

salaries['salary'] = salaries['salary'].round(2)

print(f"Salary statistics:")
print(salaries['salary'].describe())

### Method 1: IQR (Interquartile Range)

In [ ]:
# Calculate IQR
Q1 = salaries['salary'].quantile(0.25)
Q3 = salaries['salary'].quantile(0.75)
IQR = Q3 - Q1

print(f"Q1: ${Q1:,.2f}")
print(f"Q3: ${Q3:,.2f}")
print(f"IQR: ${IQR:,.2f}")

# Define outlier bounds (typically 1.5 * IQR)
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"\nOutlier bounds: ${lower_bound:,.2f} to ${upper_bound:,.2f}")

In [ ]:
# Identify outliers
outliers = salaries[(salaries['salary'] < lower_bound) | (salaries['salary'] > upper_bound)]
print(f"Found {len(outliers)} outliers:")
outliers

In [ ]:
# Remove outliers
clean_salaries = salaries[(salaries['salary'] >= lower_bound) & (salaries['salary'] <= upper_bound)]
print(f"Original: {len(salaries)} rows")
print(f"After removing outliers: {len(clean_salaries)} rows")
print(f"\nClean salary statistics:")
print(clean_salaries['salary'].describe())

### Method 2: Z-Score

In [ ]:
# Calculate Z-scores (how many standard deviations from mean)
mean = salaries['salary'].mean()
std = salaries['salary'].std()

salaries['z_score'] = (salaries['salary'] - mean) / std

# Outliers are typically |z| > 3
z_outliers = salaries[abs(salaries['z_score']) > 3]
print(f"Outliers by Z-score (|z| > 3):")
z_outliers

### Handling Outliers: Capping/Winsorizing

In [ ]:
# Instead of removing, cap outliers at bounds
salaries_capped = salaries.copy()
salaries_capped['salary_capped'] = salaries_capped['salary'].clip(lower=lower_bound, upper=upper_bound)

print("Comparison of original vs capped (showing outliers):")
salaries_capped[salaries_capped['salary'] != salaries_capped['salary_capped']][['employee_id', 'salary', 'salary_capped']]

## 7. Practical Example: Cleaning a Messy Dataset

Let's put it all together with a realistic example.

In [ ]:
# Create a messy dataset
messy_data = pd.DataFrame({
    'customer_id': ['C001', 'C002', 'C003', 'C004', 'C005', 'C006'],
    'age': ['25', '32', 'unknown', '28', '45', '150'],  # String, with invalid values
    'income': [50000, None, 65000, 70000, 'N/A', 55000],  # Missing and string
    'purchase_amount': [120.50, 89.99, 500000, 156.75, 210.00, 95.50],  # Outlier
    'email': ['alice@email.com', 'BOB@EMAIL.COM', None, 'diana@email.com', '', 'frank@email.com']
})

print("Messy data:")
messy_data

In [ ]:
# Step 1: Check initial state
print("Data types:")
print(messy_data.dtypes)
print("\nMissing values:")
print(messy_data.isnull().sum())

In [ ]:
# Step 2: Clean age column
clean_data = messy_data.copy()

# Convert age to numeric, replacing errors with NaN
clean_data['age'] = pd.to_numeric(clean_data['age'], errors='coerce')

# Replace unrealistic ages (>120) with NaN
clean_data.loc[clean_data['age'] > 120, 'age'] = None

# Fill missing ages with median
clean_data['age'] = clean_data['age'].fillna(clean_data['age'].median())

print("Age after cleaning:")
clean_data['age']

In [ ]:
# Step 3: Clean income column
# Replace 'N/A' with NaN
clean_data['income'] = clean_data['income'].replace('N/A', None)

# Convert to numeric
clean_data['income'] = pd.to_numeric(clean_data['income'], errors='coerce')

# Fill missing with median
clean_data['income'] = clean_data['income'].fillna(clean_data['income'].median())

print("Income after cleaning:")
clean_data['income']

In [ ]:
# Step 4: Handle outliers in purchase_amount
Q1 = clean_data['purchase_amount'].quantile(0.25)
Q3 = clean_data['purchase_amount'].quantile(0.75)
IQR = Q3 - Q1

upper_bound = Q3 + 1.5 * IQR

# Cap outliers
clean_data['purchase_amount'] = clean_data['purchase_amount'].clip(upper=upper_bound)

print("Purchase amount after capping:")
clean_data['purchase_amount']

In [ ]:
# Step 5: Clean email column
# Replace empty strings with NaN
clean_data['email'] = clean_data['email'].replace('', None)

# Standardize to lowercase
clean_data['email'] = clean_data['email'].str.lower()

print("Email after cleaning:")
clean_data['email']

In [ ]:
# Final result
print("Clean data:")
print(clean_data)

print("\nData types:")
print(clean_data.dtypes)

print("\nMissing values:")
print(clean_data.isnull().sum())

## Summary

In this session, we covered:

1. **concat()**: Stack DataFrames vertically or horizontally
2. **merge()**: Join DataFrames on common columns
3. **Types of joins**: inner, left, right, outer
4. **astype()**: Convert data types
5. **Missing values**: `isnull()`, `notnull()`, `dropna()`, `fillna()`
6. **Outliers**: IQR method, Z-score, capping with `clip()`

### Key Points to Remember

- Use `ignore_index=True` with concat to reset the index
- Always verify which type of join you need
- Check data types early - wrong types cause errors
- Decide whether to drop or fill missing values based on context
- Document your outlier handling decisions

### Next Session

Practice time! We'll apply these skills to clean and merge real-world datasets.